# Manipulação de Tipos de Dados Complexos no Apache Spark

O Apache Spark oferece uma variedade de funções para manipular tipos de dados complexos, como arrays, mapas e estruturas, em DataFrames. Essas funções são úteis para realizar operações de consulta, transformações e exploração de dados estruturados.

## Funções de Manipulação de Arrays

As funções a seguir são usadas para manipular arrays em DataFrames:

| Função                       | Descrição                                                   | Exemplo de Caso de Uso                   |
|------------------------------|-------------------------------------------------------------|-----------------------------------------|
| `array(col1, col2, ...)`     | Cria um array a partir de colunas específicas              | Unir colunas em um array.               |
| `size(col)`                  | Retorna o tamanho de um array                               | Contar o número de elementos em um array. |
| `array_contains(col, value)` | Verifica se um array contém um valor específico            | Filtrar registros que contenham um valor. |
| `explode(col)`               | Gera uma nova linha para cada elemento em um array          | Expandir arrays em múltiplas linhas.     |
| `concat_ws(sep, col1, col2, ...)` | Concatena elementos de um array com um separador     | Unir elementos de um array em uma string. |
| `collect_list(col)`          | Coleta elementos de uma coluna em uma lista                | Agregar dados em uma lista.             |
| `collect_set(col)`           | Coleta elementos únicos de uma coluna em um conjunto      | Agregar dados em um conjunto único.     |


In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("Exemplo").config("spark.jars.packages", "org.postgresql:postgresql:42.2.24").getOrCreate()

In [2]:
df = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/air_cia/")
)

df.printSchema()

root
 |-- razão_social: string (nullable = true)
 |-- icao_iata: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- atividades_aéreas: string (nullable = true)
 |-- endereço_sede: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- e-mail: string (nullable = true)
 |-- decisão_operacional: string (nullable = true)
 |-- data_decisão_operacional: string (nullable = true)
 |-- validade_operacional: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- iata: string (nullable = true)



In [3]:
df.select("endereço_sede").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|endereço_sede                                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA SANTOS DUMONT, KM 66, SISTEMA VIÁRIO PRINCIPAL, S/ Nº, 13.052-970, CAMPINAS-SP                                        |
|RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AEROPORTO MUNICIPAL ALBERTO BERTELLI , 86.702-670, ARAPONGAS-PR                                                         |
|AV GOVERNADOR JOÃO PONCE DE ARRUDA, S/N, HANGAR AMÉRICA DO SUL ANEXO AO AEROP. MAL RONDON. CAIXA POSTAL N° 156, NO BAIRRO CENTRO-NORTE, 78.110-900, VÁRZEA GRANDE-MT

In [13]:
df_array = (
    df
    .withColumn("endereco_array", F.split(df["endereço_sede"], ", "))
    .withColumn("len_array", F.size(F.col("endereco_array")))
    #.withColumn("explode", F.explode(F.col("endereco_array")))
    .withColumn("contem", F.array_contains(F.col("endereco_array"), "SÃO JOSÉ DOS PINHAIS-PR"))
    .withColumn("elemente1", F.col("endereco_array")[0])
)

df_array.printSchema()

# Mostrar o resultado
df_array.select("endereco_array","elemente1","len_array").show(truncate=False)

root
 |-- razão_social: string (nullable = true)
 |-- icao_iata: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- atividades_aéreas: string (nullable = true)
 |-- endereço_sede: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- e-mail: string (nullable = true)
 |-- decisão_operacional: string (nullable = true)
 |-- data_decisão_operacional: string (nullable = true)
 |-- validade_operacional: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- endereco_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- len_array: integer (nullable = false)
 |-- contem: boolean (nullable = true)
 |-- elemente1: string (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------+---------+
|endereco_


# Funções de Manipulação de Mapas

As funções a seguir são usadas para manipular mapas em DataFrames:

| Função                       | Descrição                                                   | Exemplo de Caso de Uso                   |
|------------------------------|-------------------------------------------------------------|-----------------------------------------|
| `map(col1, col2, ...)`       | Cria um mapa a partir de pares chave-valor                    | Combinar várias colunas em um mapa.      |
| `map_keys(col)`              | Retorna as chaves de um mapa                                 | Extrair as chaves de um mapa.             |
| `map_values(col)`            | Retorna os valores de um mapa                                | Extrair os valores de um mapa.           |
| `map_concat(col1, col2)`     | Concatena dois mapas em um novo mapa                         | Combinar dois mapas em um único mapa.    |
| `map_from_arrays(keys, values)` | Cria um mapa a partir de dois arrays                   | Construir um mapa a partir de colunas separadas. |
| `map_from_entries(col)`       | Cria um mapa a partir de entradas (chave, valor)           | Construir um mapa a partir de colunas de chave e valor. |

In [19]:
df_map = (
    df
    .withColumn("map_id",  F.create_map(F.lit("icao"), F.col("icao"), F.lit("iata"), F.col("iata")))
    .withColumn("map_keys",  F.map_values("map_id"))
)

df_map.printSchema()

df_map.select("map_id","map_keys","map_id.keys").show(truncate=False)

root
 |-- razão_social: string (nullable = true)
 |-- icao_iata: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- atividades_aéreas: string (nullable = true)
 |-- endereço_sede: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- e-mail: string (nullable = true)
 |-- decisão_operacional: string (nullable = true)
 |-- data_decisão_operacional: string (nullable = true)
 |-- validade_operacional: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- map_id: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- map_keys: array (nullable = false)
 |    |-- element: string (containsNull = true)

+----------------------------+------------+----+
|map_id                      |map_keys    |keys|
+----------------------------+------------+----+
|{icao -> LTG, iata -> M3}   |[LTG, M3]   |null|
|{icao -> ASO, iata -> 2S}   |[ASO, 2S]   |null|
|{icao -> SUL, iata -> 0A}  

# Funções de Manipulação de Estruturas

As funções a seguir são usadas para manipular estruturas em DataFrames:

| Função                           | Descrição                                                 | Exemplo de Caso de Uso                         |
|----------------------------------|-----------------------------------------------------------|-----------------------------------------------|
| `struct(col1, col2, ...)`        | Cria uma estrutura a partir de colunas específicas       | Combinar várias colunas em uma estrutura.      |
| `struct_fields(col)`             | Retorna os campos de uma estrutura                        | Extrair os campos de uma estrutura.             |
| `named_struct(fields)`           | Cria uma estrutura nomeada com campos especificados      | Criar uma estrutura com campos nomeados.        |
| `getField(col, fieldName)`       | Retorna o valor de um campo em uma estrutura              | Acessar valores de campos em uma estrutura.     |

In [26]:
df_map = (
    df
    .withColumn("struct_info",  F.struct("atividades_aéreas","cnpj","telefone"))
    .withColumn("column_struct", F.col("struct_info")["atividades_aéreas"])
)

df_map.printSchema()

df_map.select("column_struct","column_struct").show(truncate=False)

root
 |-- razão_social: string (nullable = true)
 |-- icao_iata: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- atividades_aéreas: string (nullable = true)
 |-- endereço_sede: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- e-mail: string (nullable = true)
 |-- decisão_operacional: string (nullable = true)
 |-- data_decisão_operacional: string (nullable = true)
 |-- validade_operacional: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- struct_info: struct (nullable = false)
 |    |-- atividades_aéreas: string (nullable = true)
 |    |-- cnpj: string (nullable = true)
 |    |-- telefone: string (nullable = true)
 |-- column_struct: string (nullable = true)

+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+
|column_struct                               